<a href="https://colab.research.google.com/github/matteo9903/FCC/blob/main/Data%20Preprocessing%20and%20Embeddings/Text_Classification_using_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Data Link: ](https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews)

**riutilizzo dataset IMDb coi 50k film per applicare sentiment analysis**<br><br>
finora ***non si stanno utlizzando LLM, ma solo semplice ML***

In [58]:
!pip install Keras-Preprocessing
# per evitare ModuleNotFoundError: No module named 'keras.preprocessing.text'


In [59]:
# IMPORTO TUTTE LE LIBRERIE NECESSARIE
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# PER ORA LIBRERIE DL NON NECESSARIE
# from keras.models import Model
# from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, SpatialDropout1D
# from keras.optimizers import RMSprop
# from tensorflow.keras.preprocessing.text import Tokenizer # ho dovuto aggiungere tensorflow. davanti
# from keras.preprocessing import sequence
# from keras.utils import to_categorical
# from keras.callbacks import EarlyStopping, ModelCheckpoint
# from keras.models import Sequential
# from keras.utils import pad_sequences


from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import confusion_matrix
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 255)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [60]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [61]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [62]:
%cd /content/drive/MyDrive/CORSO_GENAI/PREPROCESSING/data

/content/drive/MyDrive/CORSO_GENAI/PREPROCESSING/data


In [63]:
# IMPORTO DATASET
data_path = "IMDB Dataset.csv"
df = pd.read_csv(data_path)
df.head()

,review,sentiment
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of v...",positive
1,"A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen-...",positive
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well b...",positive
3,"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zombie.<br...",negative
4,"Petter Mattei's ""Love in the Time of Money"" is a visually stunning film to watch. Mr. Mattei offers us a vivid portrait about human relations. This is a movie that seems to be telling us what money, power and success do to people in the different situ...",positive


In [64]:
df.shape

(50000, 2)

In [65]:
# considero per semplicità solo i primi 10000
df = df.iloc[:10000]
df.head()

,review,sentiment
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of v...",positive
1,"A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen-...",positive
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well b...",positive
3,"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zombie.<br...",negative
4,"Petter Mattei's ""Love in the Time of Money"" is a visually stunning film to watch. Mr. Mattei offers us a vivid portrait about human relations. This is a movie that seems to be telling us what money, power and success do to people in the different situ...",positive


In [66]:
df.shape

(10000, 2)

In [67]:
df['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [68]:
df['sentiment'].value_counts()

# conto sentimenti (= output) positivi e negativi
# notare che è abbastanza bilanciato

,count
sentiment,
positive,5028
negative,4972


In [69]:
# verifico valori nulli --> che non ci sono
df.isnull().sum()

,0
review,0
sentiment,0


In [70]:
# verifico presenza duplicati --> ci sono
df.duplicated().sum()

17

In [71]:
# rimuovo duplicati
df.drop_duplicates(inplace=True)

In [72]:
df.duplicated().sum()

0

# Basic Preprocessing
  - Remove tags - HTML
  - Lower case
  - remove stopwords

In [73]:
# rimuovo tag HTML
import re
def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'), '', raw_text)
    return cleaned_text

In [74]:
df['review'] = df['review'].apply(remove_tags)

In [75]:
df.head()

,review,sentiment
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.The first thing that struck me about Oz was its brutality and unflinching scenes of violence, whi...",positive
1,"A wonderful little production. The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. The actors are extremely well chosen- Michael Sheen not only ...",positive
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well b...",positive
3,"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zombie.OK, first of al...",negative
4,"Petter Mattei's ""Love in the Time of Money"" is a visually stunning film to watch. Mr. Mattei offers us a vivid portrait about human relations. This is a movie that seems to be telling us what money, power and success do to people in the different situ...",positive


In [76]:
df['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.I would say the main appeal of the show is due to the fact that it goes where other shows wo

In [77]:
# lower case di tutte òe recensioni
df['review'] = df['review'].apply(lambda x:x.lower())

In [78]:
df['review'][0]

"one of the other reviewers has mentioned that after watching just 1 oz episode you'll be hooked. they are right, as this is exactly what happened with me.the first thing that struck me about oz was its brutality and unflinching scenes of violence, which set in right from the word go. trust me, this is not a show for the faint hearted or timid. this show pulls no punches with regards to drugs, sex or violence. its is hardcore, in the classic use of the word.it is called oz as that is the nickname given to the oswald maximum security state penitentary. it focuses mainly on emerald city, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. em city is home to many..aryans, muslims, gangstas, latinos, christians, italians, irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.i would say the main appeal of the show is due to the fact that it goes where other shows wo

In [79]:
# RIMUOVO STOPWORDS
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')
sw_list = stopwords.words('english')

df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


essendo una task di ***sentiment analysis***, si possono rimuovere senza problemi le stopwords perché mi interessano solo le parole con significato positivo/negativo

In [80]:
df['review'][0]

"one reviewers mentioned watching 1 oz episode hooked. right, exactly happened me.the first thing struck oz brutality unflinching scenes violence, set right word go. trust me, show faint hearted timid. show pulls punches regards drugs, sex violence. hardcore, classic use word.it called oz nickname given oswald maximum security state penitentary. focuses mainly emerald city, experimental section prison cells glass fronts face inwards, privacy high agenda. em city home many..aryans, muslims, gangstas, latinos, christians, italians, irish more....so scuffles, death stares, dodgy dealings shady agreements never far away.i would say main appeal show due fact goes shows dare. forget pretty pictures painted mainstream audiences, forget charm, forget romance...oz mess around. first episode ever saw struck nasty surreal, say ready it, watched more, developed taste oz, got accustomed high levels graphic violence. violence, injustice (crooked guards who'll sold nickel, inmates who'll kill order g

In [81]:
df.head()

,review,sentiment
0,"one reviewers mentioned watching 1 oz episode hooked. right, exactly happened me.the first thing struck oz brutality unflinching scenes violence, set right word go. trust me, show faint hearted timid. show pulls punches regards drugs, sex violence. ha...",positive
1,"wonderful little production. filming technique unassuming- old-time-bbc fashion gives comforting, sometimes discomforting, sense realism entire piece. actors extremely well chosen- michael sheen ""has got polari"" voices pat too! truly see seamless edit...",positive
2,"thought wonderful way spend time hot summer weekend, sitting air conditioned theater watching light-hearted comedy. plot simplistic, dialogue witty characters likable (even well bread suspected serial killer). may disappointed realize match point 2: r...",positive
3,"basically there's family little boy (jake) thinks there's zombie closet & parents fighting time.this movie slower soap opera... suddenly, jake decides become rambo kill zombie.ok, first going make film must decide thriller drama! drama movie watchable...",negative
4,"petter mattei's ""love time money"" visually stunning film watch. mr. mattei offers us vivid portrait human relations. movie seems telling us money, power success people different situations encounter. variation arthur schnitzler's play theme, director ...",positive


In [108]:
# separo input da output
X = df.iloc[:,0:1]
y = df['sentiment']

In [121]:
X.head()

,review
0,"one reviewers mentioned watching 1 oz episode hooked. right, exactly happened me.the first thing struck oz brutality unflinching scenes violence, set right word go. trust me, show faint hearted timid. show pulls punches regards drugs, sex violence. ha..."
1,"wonderful little production. filming technique unassuming- old-time-bbc fashion gives comforting, sometimes discomforting, sense realism entire piece. actors extremely well chosen- michael sheen ""has got polari"" voices pat too! truly see seamless edit..."
2,"thought wonderful way spend time hot summer weekend, sitting air conditioned theater watching light-hearted comedy. plot simplistic, dialogue witty characters likable (even well bread suspected serial killer). may disappointed realize match point 2: r..."
3,"basically there's family little boy (jake) thinks there's zombie closet & parents fighting time.this movie slower soap opera... suddenly, jake decides become rambo kill zombie.ok, first going make film must decide thriller drama! drama movie watchable..."
4,"petter mattei's ""love time money"" visually stunning film watch. mr. mattei offers us vivid portrait human relations. movie seems telling us money, power success people different situations encounter. variation arthur schnitzler's play theme, director ..."


In [110]:
y

,sentiment
0,positive
1,positive
2,positive
3,negative
4,positive
5,positive
6,positive
7,negative
8,negative
9,positive


In [111]:
# utilizzo LabelEncoder per associare 1 e 0 a output positivi e negativi
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

y = encoder.fit_transform(y)

In [112]:
y

array([1, 1, 1, ..., 0, 0, 1])

In [87]:
# faccio split in training e test set (proporzione 80-20)
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [88]:
X_train.shape

(7986, 1)

In [89]:
X_test.shape

(1997, 1)

#### Bag of Words technique

In [33]:
# Applying BoW
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

In [34]:
# faccio training del BoW su training e poi lo riapplico sul test set
X_train_bow = cv.fit_transform(X_train['review']).toarray()
X_test_bow = cv.transform(X_test['review']).toarray()

In [35]:
X_train_bow

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [46]:
# applico modello ML
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
# fit del modello
gnb.fit(X_train_bow,y_train)

GaussianNB()

In [47]:
y_pred = gnb.predict(X_test_bow)

from sklearn.metrics import accuracy_score,confusion_matrix
accuracy_score(y_test,y_pred)

# accuratezza del 63,2% con Naive Bayes

0.6324486730095142

In [48]:
confusion_matrix(y_test,y_pred)

array([[717, 235],
       [499, 546]])

In [49]:
# APPLICO RANDOM FOREST
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

rf.fit(X_train_bow,y_train)

y_pred = rf.predict(X_test_bow)

accuracy_score(y_test,y_pred)

# accuratezza dell'84%, buona

0.8432648973460191

In [50]:
# utilizzo non tutte le parole, ma solo le 3000 che compaiono di più, non mi
# servono tutte, per non rendere modello troppo complesso o fare overfitting
cv = CountVectorizer(max_features=3000)  # creo spazio 3000-D

X_train_bow = cv.fit_transform(X_train['review']).toarray()
X_test_bow = cv.transform(X_test['review']).toarray()

rf = RandomForestClassifier()

rf.fit(X_train_bow,y_train)
y_pred = rf.predict(X_test_bow)
accuracy_score(y_test,y_pred)

# accuratezza buona, simile al caso con titte le features ma con molto meno
# costo computazionale

0.8352528793189785

#### N grams

In [51]:
cv = CountVectorizer(ngram_range=(2,2),max_features=5000)

# ngrame_range = (n_min, n_max) --> mettendo (2,2) uso solo bigrams
                            #   --> mettendo (1,2) uso sia bigram che unigram

X_train_bow = cv.fit_transform(X_train['review']).toarray()
X_test_bow = cv.transform(X_test['review']).toarray()

rf = RandomForestClassifier()

rf.fit(X_train_bow,y_train)
y_pred = rf.predict(X_test_bow)
accuracy_score(y_test,y_pred)

# accuratezza risulta minure rispetto a semplice BoW, efficacia di uno o dello
# altro metodo dipendono dal contesto applicativo e sperimentale

0.7526289434151227

#### TFIDF

In [52]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

In [53]:
X_train_tfidf = tfidf.fit_transform(X_train['review']).toarray()
X_test_tfidf = tfidf.transform(X_test['review'])

In [54]:
rf = RandomForestClassifier()

rf.fit(X_train_tfidf,y_train)
y_pred = rf.predict(X_test_tfidf)

accuracy_score(y_test,y_pred)

# accuratezza buona, simile a caso BoW

0.8462694041061593

in questo caso accuratezza è dell'84%, buona ma non eccellente, ma perché si sono applicati modelli di semplice ML.<br>
se ***si applicassero modelli DL*** si otterrebbero ottimi risultati, ipoteticamente *superiori al 90%*

#### word2vec

In [90]:
import gensim
import os

In [91]:
!pip install --upgrade gensim --user # aggiorno ad ultima versione

In [44]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [122]:
text = []
#raw_sent = sent_tokenize(corpus)
# questo primo passaggio non serve perché ho già le recensioni suddivise nelle
# righe del dataset; non ho un intero testo da separare in frasi
# - riciclo dataset già processato per creare l'array da dare in pasto a word2vec
for index, row in X.iterrows():
    text.append(simple_preprocess(row['review']))

In [124]:
text[0:10]

[['one',
  'reviewers',
  'mentioned',
  'watching',
  'oz',
  'episode',
  'hooked',
  'right',
  'exactly',
  'happened',
  'me',
  'the',
  'first',
  'thing',
  'struck',
  'oz',
  'brutality',
  'unflinching',
  'scenes',
  'violence',
  'set',
  'right',
  'word',
  'go',
  'trust',
  'me',
  'show',
  'faint',
  'hearted',
  'timid',
  'show',
  'pulls',
  'punches',
  'regards',
  'drugs',
  'sex',
  'violence',
  'hardcore',
  'classic',
  'use',
  'word',
  'it',
  'called',
  'oz',
  'nickname',
  'given',
  'oswald',
  'maximum',
  'security',
  'state',
  'penitentary',
  'focuses',
  'mainly',
  'emerald',
  'city',
  'experimental',
  'section',
  'prison',
  'cells',
  'glass',
  'fronts',
  'face',
  'inwards',
  'privacy',
  'high',
  'agenda',
  'em',
  'city',
  'home',
  'many',
  'aryans',
  'muslims',
  'gangstas',
  'latinos',
  'christians',
  'italians',
  'irish',
  'more',
  'so',
  'scuffles',
  'death',
  'stares',
  'dodgy',
  'dealings',
  'shady',
  'ag

In [123]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2
)

# converto in vettore con "build_vocab"
model.build_vocab(text)

# faccio training del modello, model.corpus_count e model.epochs sono parametri
# di default ma possono essere cambiati
model.train(text, total_examples=model.corpus_count, epochs=model.epochs)

(5875194, 6212140)

In [125]:
# noto che ho 31845 vettori, ovvero 31845 parole
vec = model.wv.get_normed_vectors()  # ottengo vettori normalizzati
vec

array([[-0.00353938,  0.0334816 ,  0.07578875, ..., -0.11976386,
         0.02328813, -0.11446781],
       [-0.09629512,  0.11845125,  0.02944383, ...,  0.11793696,
        -0.06356744, -0.02138858],
       [ 0.01835281,  0.08915452,  0.1441938 , ..., -0.09671647,
        -0.00999538, -0.08914141],
       ...,
       [-0.04399985,  0.11568843,  0.06481659, ..., -0.10987151,
        -0.06817022,  0.02128803],
       [-0.08058675,  0.07354615,  0.02720006, ..., -0.07057694,
        -0.05339149,  0.0996706 ],
       [ 0.04078857, -0.02358251,  0.04341565, ..., -0.18787359,
        -0.00963419,  0.01967815]], dtype=float32)

In [126]:
# estraggo indice di ogni singola parola/vettore
y_w2v = model.wv.index_to_key
y_w2v.index('time')

8

In [129]:
# ora devo associare ad ogni parola all'interno di una recensione il corrispondente vettore, e poi fare
# la media dei vettori all'interno di ogni recensione per avere una misura adatta ad effettuare la
# predizione (n_vettori = n_recensioni)
vector_text = []
for index, row in enumerate(text):
    vector_text.append([])
    for i, word in enumerate(row):
        if word in y_w2v:
          y_ind = y_w2v.index(word)
          vector_text[index].append(vec[y_ind])

(vector_text[0:10])

[[array([ 0.01835281,  0.08915452,  0.1441938 , -0.06532823,  0.08667494,
         -0.01224875, -0.06503853,  0.16349162, -0.19398212, -0.08754376,
          0.02461754, -0.05623944,  0.10643795,  0.22548775,  0.13465668,
         -0.05378996,  0.08240291,  0.03850638, -0.1059069 , -0.17960511,
         -0.03921441, -0.03371869, -0.11148536, -0.07915101, -0.07446272,
         -0.03655426,  0.1371444 ,  0.10317136, -0.02461462,  0.08128539,
          0.07068567, -0.05067717, -0.01461861, -0.1433547 , -0.00737262,
          0.07557752,  0.05125007, -0.11171916, -0.0341105 , -0.05421112,
         -0.16517389, -0.00144548,  0.09337483,  0.10694922,  0.06747381,
          0.06053891,  0.14626528, -0.1033569 ,  0.12403542, -0.11950877,
         -0.06846773, -0.06084128, -0.25806984, -0.13111845, -0.11808229,
         -0.06987779, -0.0059583 , -0.04578559, -0.05431322, -0.17169926,
          0.07024599, -0.02116082,  0.21607387,  0.02306194, -0.10705237,
          0.04827228, -0.02788453,  0.

In [131]:
len(vector_text)  # = a numero recensioni

9983

In [135]:
# calcolo media vettori in ogni recensione codificata
mean_vector_text = []
for index, row in enumerate(vector_text):
    mean_vector_text.append(np.mean(row, axis=0))

mean_vector_text[0:10]

[array([-0.03337327,  0.09333178,  0.03908387,  0.02268611, -0.01737296,
        -0.15448974,  0.03010466,  0.24363439, -0.05422153, -0.06447774,
        -0.07763012, -0.12254394, -0.01811733,  0.02084697,  0.04595002,
        -0.0172602 ,  0.00980221, -0.07206351,  0.00834794, -0.1464791 ,
         0.02579465,  0.06070441,  0.03261267, -0.05605239, -0.06656041,
         0.00907675, -0.06593459, -0.02640109, -0.0851868 , -0.00694276,
         0.07992951,  0.00548804,  0.04108743, -0.04335022, -0.04975106,
         0.11470956,  0.01435317, -0.07683929, -0.04646957, -0.18636307,
         0.04485842, -0.0692163 , -0.00658168, -0.03772764,  0.12615263,
        -0.02835018, -0.0668539 , -0.01635552,  0.03211574,  0.07510087,
         0.02763497, -0.10123123, -0.07374752, -0.02784913, -0.0536302 ,
         0.04423906,  0.01465536, -0.01640922, -0.07168493,  0.01334039,
         0.0229709 ,  0.02907693,  0.01248204,  0.00562042, -0.10827249,
         0.08390845,  0.01904061,  0.02593214, -0.0

In [137]:
vec_train,vec_test,y_train,y_test = train_test_split(mean_vector_text,y,test_size=0.2,random_state=1)
len(vec_train)

7986

In [139]:
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

rf.fit(vec_train,y_train)
y_pred = rf.predict(vec_test)

accuracy_score(y_test,y_pred)
# accuratezza minore rispetto che ad altri metodi = 81.6%

0.8162243365047571

In [140]:
confusion_matrix(y_test,y_pred)


array([[761, 191],
       [176, 869]])